# CS 145 COVID Prediction

## Loading Data

First we need to load the data, which is stored in CSV files separated by date

In [40]:
import numpy as np
import pandas as pd
import glob
from sklearn.impute import KNNImputer

In [159]:
# grab filepaths for every csv file
datapath = '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/*.csv'
datafiles = np.array(glob.glob(datapath))
print(datafiles[138:142])

['../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-28-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-29-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-30-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-31-2020.csv']


In [160]:
# init giant df
data = pd.read_csv(datafiles[0])
# read and append each csv to the master df
for i in range(1, len(datafiles)):
    df = pd.read_csv(datafiles[i])
    data = data.append(df, ignore_index = True)
# only keep 50 states
data.drop(data[data.Province_State.isin(['Diamond Princess', 'District of Columbia', 'Grand Princess', 'Guam', 'Puerto Rico', 'American Samoa', 'Northern Mariana Islands', 'Recovered', 'Virgin Islands'])].index, inplace = True)
data.drop(columns = ['Country_Region', 'FIPS', 'UID', 'ISO3'], inplace = True)
data['Last_Update'] = data['Last_Update'].astype('datetime64[ns]')
data = data.reset_index(drop=True)
data.loc[48:52, ]

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
48,Wisconsin,2020-04-12 23:18:15,44.2685,-89.6165,3341,144,NaN,3197.0,64.565739,39257.0,974.0,4.310087,758.652263,29.152948
49,Wyoming,2020-04-12 23:18:15,42.7560,-107.3025,270,0,137.0,270.0,54.299735,5459.0,40.0,0.000000,1097.860188,14.814815
50,Alabama,2020-04-13 23:07:54,32.3182,-86.9023,3734,99,NaN,3635.0,79.634933,29182.0,457.0,2.651312,622.363852,12.238886
51,Alaska,2020-04-13 23:07:54,61.3707,-152.4044,277,8,85.0,269.0,46.340521,7830.0,32.0,2.888087,1309.914362,11.552347
52,Arizona,2020-04-13 23:07:54,33.7298,-111.4312,3705,122,NaN,3583.0,50.901828,43347.0,525.0,3.292848,595.530778,14.170040


## Data Cleaning

In [161]:
import datetime
# add a column for days since Jan 1st(will eventually remove timestamp)
# too lazy to vectorize
days = np.zeros(len(data.index))
for i in range(len(data.index)):
    days[i] = (data['Last_Update'][i].date() - datetime.date(2020, 1, 1)).days
data['days'] = days

In [162]:
data = pd.concat([data, pd.get_dummies(data['Province_State'])], axis = 1)
data.drop(columns = ['Province_State'], inplace = True)
names = list(data.columns.values)
timestamps = data['Last_Update']
data.head()

,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2020-04-12 23:18:15,32.3182,-86.9023,3563,93,NaN,3470.0,75.988020,21583.0,437.0,...,0,0,0,0,0,0,0,0,0,0
1,2020-04-12 23:18:15,61.3707,-152.4044,272,8,66.0,264.0,45.504049,8038.0,31.0,...,0,0,0,0,0,0,0,0,0,0
2,2020-04-12 23:18:15,33.7298,-111.4312,3542,115,NaN,3427.0,48.662422,42109.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2020-04-12 23:18:15,34.9697,-92.3731,1280,27,367.0,1253.0,49.439423,19722.0,130.0,...,0,0,0,0,0,0,0,0,0,0
4,2020-04-12 23:18:15,36.1162,-119.6816,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
imputer = KNNImputer(n_neighbors=10, weights="uniform")
data = pd.DataFrame(data = imputer.fit_transform(data.drop(columns = ['Last_Update'])), columns = names[1:])
data = pd.concat([timestamps, data], axis = 1)
data.head()

,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2020-04-12 23:18:15,32.3182,-86.9023,3563.0,93.0,780.3,3470.0,75.988020,21583.0,437.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-04-12 23:18:15,61.3707,-152.4044,272.0,8.0,66.0,264.0,45.504049,8038.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-04-12 23:18:15,33.7298,-111.4312,3542.0,115.0,1378.2,3427.0,48.662422,42109.0,735.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-04-12 23:18:15,34.9697,-92.3731,1280.0,27.0,367.0,1253.0,49.439423,19722.0,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-04-12 23:18:15,36.1162,-119.6816,22795.0,640.0,4283.8,22155.0,58.137726,190328.0,5234.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train = data.drop(data[data.Last_Update.astype('datetime64[ns]') > '2020-09-01'].index)
train.drop(columns = "Last_Update", inplace = True)
print(train.head())
test = data.drop(data[(data.Last_Update.astype('datetime64[ns]') <= '2020-09-01')].index)
test.drop(test[(test.Last_Update.astype('datetime64[ns]') > '2020-09-27')].index, inplace = True)
test.drop(columns = "Last_Update", inplace = True)
print(test.head())

In [157]:
cut = (datetime.date(2020, 9, 1) - datetime.date(2020, 1, 1)).days
# training data must be before Sept. 1st
train = data.drop(data[data.days >= cut].index)
# testing data is during Sept 1-26th
cut2 = (datetime.date(2020, 9, 26) - datetime.date(2020, 1, 1)).days
test = data.drop(data[(data.days < cut) & (data.days > cut2)].index)
print(test['days'])

0       102.0
1       102.0
2       102.0
3       102.0
4       102.0
        ...  
9645    295.0
9646    295.0
9647    295.0
9648    295.0
9649    295.0
Name: days, Length: 9650, dtype: float64


In [61]:
data['Last_Update'].astype('datetime64[ns]') < '2020-05-01'

0        True
1        True
2        True
3        True
4        True
        ...  
7145    False
7146    False
7147    False
7148    False
7149    False
Name: Last_Update, Length: 7150, dtype: bool

In [48]:
from sklearn.linear_model import LogisticRegression